# Cross-filtering

[![open_in_colab][colab_badge]][colab_notebook_link]
<!-- [![open_in_binder][binder_badge]][binder_notebook_link] -->

[colab_badge]: https://colab.research.google.com/assets/colab-badge.svg
[colab_notebook_link]: https://colab.research.google.com/github/foursquare/fsq-studio-sdk-examples/blob/master/python-notebooks/06%20-%20Crossfilter.ipynb
<!-- [binder_badge]: https://mybinder.org/badge_logo.svg
[binder_notebook_link]: https://mybinder.org/v2/gh/foursquare/fsq-studio-sdk-examples/master?urlpath=lab/tree/python-notebooks/06%20-%20Crossfilter.ipynb -->

Cross-filtering is a technique often used in dashboards where filters are synced across multiple views which represent different facets of the same data. This example shows how Studio map filters can be coordinated with Plotly charts.

## Dependencies

This notebook requires the following Python dependencies:

- `foursquare.map-sdk`: The Studio Map SDK
- `pandas`: DataFrame library
- `plotly`: graph plotting library

If running this notebook in Binder, these dependencies should already be installed. If running in Colab, the next cell will install these dependencies.

In [ ]:
# If in Colab, install this notebook's required dependencies
import sys
if "google.colab" in sys.modules:
    !pip install 'foursquare.map_sdk>=3.0.1' pandas 'plotly>=5.10.0'

## Imports

In [ ]:
import foursquare.map_sdk as map_sdk
import pandas as pd
import plotly.graph_objects as go
from uuid import uuid4

## Syncing a map filter with a Plotly histogram

Let's create a local map and add data to it:

In [ ]:
map = map_sdk.create_map(
  api_key="<your-api-key>"
)
map

In [ ]:
url = 'https://4sq-studio-public.s3.us-west-2.amazonaws.com/sdk/examples/sample-data/earthquakes.csv'
df = pd.read_csv(url)

In [ ]:
dataset_id = str(uuid4())
map.add_dataset(map_sdk.LocalDatasetCreationProps(
    id=dataset_id,
    label='Earthquakes',
    data=df
))

Let's add a `Magnitude` filter to the map:

In [ ]:
map.add_filter(map_sdk.PartialRangeFilter(
        id='magnitude_filter',
        sources=[map_sdk.PartialFilterSource(
            data_id=dataset_id,
            field_name='Magnitude'
        )],
        value=(df['Magnitude'].min(), df['Magnitude'].max())
))

We can use Plotly to draw a histogram showing the distribution of the numbers of earthquakes by their magnitude:

In [ ]:
hist_fig = go.FigureWidget([go.Histogram(x = df['Magnitude'], nbinsx = 50)])
hist = hist_fig.data[0]

Let's now add event handlers. We supply an `on_selection` function that will be called when the selection in the Plotly histogram is changed:

In [ ]:
def on_histogram_selection_change(trace, points, state):
    # Update the Magnitude filter in the map
    map.update_filter(
        'magnitude_filter',
        value=map_sdk.PartialRangeFilter(value=(min(points.xs), max(points.xs)))
    )
hist_fig.data[0].on_selection(on_histogram_selection_change)

The `on_selection` event handler will be called when a filter in the Studio map is changed (refer to the [Map SDK docs](https://docs.unfolded.ai/map-sdk/api/set-map-event-handlers) for more info on map event handling):

In [ ]:
def on_map_filter_change(event_info):
    # Update the selection in the histogram
    if 'magnitude_filter' in event_info['id']:
        v = event_info['value']
        hist_fig.update_selections(dict(x0=v[0], x1=v[1],y0=0, y1=9000))

map.set_event_handlers(map_sdk.EventHandlers(
    on_filter_update=on_map_filter_change
))

In [ ]:
hist_fig.update_layout(
    title = 'Earthquakes by magnitude',
    height = 350,
    xaxis = {'title': 'Magnitude'},
    yaxis = {'title': 'Count'},
    dragmode = 'select',
    hovermode = 'closest'
)
hist_fig

Let's add an initial selection. It appears this is necessary for the Plotly API to connect properly

In [ ]:
if len(list(hist_fig.select_selections())) == 0:
    hist_fig.add_selection(x0=4, x1=5, y0=0, y1=5000)

You can now open the left side bar in the map and select **Filters** in the top nav menu:

<img src="https://raw.githubusercontent.com/foursquare/fsq-studio-sdk-examples/master/notebooks/images/studio-filters-pane-2.jpg" width=350>

Try changing the selection in the Plotly histogram by dragging (make sure you are using the **Box Select** tool). You should see the Magnitude filter update in the map when you change the selection in the Plotly histogram and vice versa:

<img src="https://raw.githubusercontent.com/foursquare/fsq-studio-sdk-examples/master/notebooks/images/crossfilter-480.gif" width=480>

## Scatterplot

Let's now do the same with a more sophisticated chart, a scatterplot:

In [ ]:
scatter_fig = go.FigureWidget([
    go.Scattergl(
        x = df['Magnitude'],
        y = df['Depth'],
        marker = {'color': df['Depth'], 'size': df['Magnitude']},
        mode = 'markers',
        unselected = {'marker': {'color':'rgb(200,200, 200)', 'opacity':0.9}},
    )
])

Let's add a `Depth` filter to the map

In [ ]:
map.add_filter(map_sdk.PartialRangeFilter(
    id='depth_filter',
    sources=[map_sdk.PartialFilterSource(
            data_id=dataset_id,
            field_name='Depth'
        )],
    value=(df['Depth'].min(), df['Depth'].max())
))

Let's again add event handlers. This function will be called when the selection in the scatterplot is changed:

In [ ]:
depth_filter = [df['Depth'].min(), df['Depth'].max()]
magnitude_filter = [df['Magnitude'].min(), df['Magnitude'].max()]

def on_scatterplot_selection_change(trace, points, state):
    # These need to be declared as global so that they
    # refer to the global scope variables defined above
    global magnitude_filter
    global depth_filter
    magnitude_filter = (
        min(points.xs, default = df['Magnitude'].min()),
        max(points.xs, default = df['Magnitude'].max())
    )
    depth_filter = (
        min(points.ys, default = df['Depth'].min()),
        max(points.ys, default = df['Depth'].max())
    )
    map.update_filter(
        'magnitude_filter',
        map_sdk.PartialRangeFilter(value=magnitude_filter)
    )
    map.update_filter(
        'depth_filter',
        map_sdk.PartialRangeFilter(value=depth_filter)
    )
scatter_fig.data[0].on_selection(on_scatterplot_selection_change)

Note that we are now setting two filters: one for `Magnitude`, another for `Depth`.

Let's add a map filter event handler:

In [ ]:
def on_map_filter_change2(event_info):
    global magnitude_filter
    global depth_filter
    if 'magnitude_filter' in event_info['id']:
        v = event_info['value']
        magnitude_filter = v
    if 'depth_filter' in event_info['id']:
        v = event_info['value']
        depth_filter = v
    scatter_fig.update_selections(dict(
        x0=magnitude_filter[0],
        x1=magnitude_filter[1],
        y0=depth_filter[0],
        y1=depth_filter[1]
    ))

# This will overwrite the previously set map on_filter event handler
map.set_event_handlers(map_sdk.EventHandlers(
    on_filter_update=on_map_filter_change2
))

Let's now render the scatterplot:

In [ ]:
scatter_fig.update_layout(
    title = 'Earthquakes by depth and magnitude',
    width = 700,
    height = 500,
    xaxis = {'title': 'Magnitude'},
    yaxis = {'title': 'Depth'},
    dragmode = 'select',
    hovermode = 'closest'
)
scatter_fig

Let's add an initial selection again:

In [ ]:
if len(list(scatter_fig.select_selections())) == 0:
    scatter_fig.add_selection(x0=4, x1=5, y0=20, y1=80)

You can interactively select a subset of the data in the Plotly scatterplot (use the **Box Select** tool in the  Scatterplot chart toolbar, **Lasso Select** doesn't sync correctly yet). You should see the Magnitude and Depth filters automatically update in the map when you change the selection in the scatterplot.

Likewise, if you change one of the filters in the map, you should see the selection update in the Plotly scatterplot.